In [2]:
import pandas as pd 
import numpy as np
import os
import sys
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

DEFAULT_TODAY = (datetime.today() - timedelta(days=365)).strftime('%Y%m%d')
DEFAULT_LASTYEAR = datetime.today().strftime('%Y%m%d')

os.chdir(sys.path[0] + "\\..")
os.getcwd()

from pysrc import utils


In [3]:
RATING_WINDOW = 30

PREDICT_WINDOW = 7
prev_d = 5 * PREDICT_WINDOW // 7


In [6]:
dirPath = "data/raw/"
tagDirPath = dirPath + "sohu_stock/"
data_date_lst = [] 
for filename in os.listdir(tagDirPath):
    if os.path.isdir(tagDirPath + filename):
        data_date_lst.append(datetime.strptime(filename, '%Y-%m-%d').date())

date = max(data_date_lst).strftime('%Y-%m-%d')
tag_df = pd.read_csv(tagDirPath + date + ".csv")

tag_df["stockCode"] = tag_df['stockCode'].astype("str").apply(lambda x: x.zfill(6)).to_list()
tag_df[~tag_df['sector'].isin(["央视50_","AH股","分拆预期","破净股","证金持股","上证50_","HS300_","上证180_","中证500","深成500","深证100R","标准普尔"])]
tag_df.drop(['date'],axis=1,inplace=True)
tag_df

,stockCode,stockName,sector,sectorCount
0,600337,美克家居,2025规划,81
1,300720,海川智能,2025规划,81
2,000810,创维数字,2025规划,81
3,603111,康尼机电,2025规划,81
4,300729,乐歌股份,2025规划,81
...,...,...,...,...
63948,300900,广联航空,黑龙江板块,41
63949,600853,龙建股份,黑龙江板块,41
63950,301371,敷尔佳,黑龙江板块,41
63951,688459,哈铁科技,黑龙江板块,41


### Rating

In [8]:
rating_df = pd.read_csv("个股研报.csv")
rating_df['publishDate'] = pd.to_datetime(rating_df['publishDate']).dt.strftime('%Y-%m-%d')
rating_df["stockCode"] = rating_df['stockCode'].astype("str").apply(lambda x: x.zfill(6)).to_list()
rating_df.drop(['title','orgName','authorID'], axis=1,inplace=True)

# Filter based on tag_df 
stock_code_list = tag_df['stockCode'].to_list()
rating_df = rating_df[rating_df['stockCode'].isin(stock_code_list)]
rating_df = pd.merge(tag_df, rating_df, how='inner',on=['stockCode','stockName'])
# rating_df


In [682]:
rating_df['date'] = pd.to_datetime(rating_df['publishDate'])

rating_df.sort_values(by=['sector', 'date'], inplace=True)
df = rating_df
window_size = 30

date_df = price_df.reset_index().rename(columns={"日期":"date"})[['date']].drop_duplicates()
date_df['date'] = pd.to_datetime(date_df['date'])

grouped = df.groupby('sector')

# Initialize lists to store results
total_stock_count = []
increase_from_last_week = []
increase_from_two_weeks_ago = []
weighted_score_avg = []

weekly_rating_df = pd.DataFrame()

# Loop through each sector
for sector, sector_data in grouped:
    # Calculate rolling statistics
    rolling_count = sector_data.groupby(['date','stockCode','orgCode']).size().groupby('date').count().reset_index().rename(columns={0:"count"})
    rolling_count = pd.merge(date_df,rolling_count,how='outer',on='date').sort_values('date').fillna(0)
    rolling_count = rolling_count.set_index('date').rolling(window=window_size,min_periods=1).sum()
    rolling_count = rolling_count[rolling_count.index.weekday==4]

    rolling_weighted_score_avg = sector_data.groupby(['date'])['emRatingValue'].apply(sum).reset_index()
    rolling_weighted_score_avg = pd.merge(date_df,rolling_weighted_score_avg,how='outer',on='date').sort_values('date').fillna(0)
    rolling_weighted_score_avg = rolling_weighted_score_avg.set_index('date').rolling(window=window_size,min_periods=1).apply(lambda x:np.exp(np.linspace(-5, 0, window_size))[-len(x):].dot(x))
    rolling_weighted_score_avg = rolling_weighted_score_avg[rolling_weighted_score_avg.index.weekday==4]
   
    # Create a DataFrame for the sector's rolling statistics
    sector_results = pd.DataFrame({
        'sector': sector,
        'end_of_week_date': rolling_count.index.get_level_values('date'),
        'count': rolling_count['count'],
        'weighted_score_avg': rolling_weighted_score_avg['emRatingValue']
    })

    # Append the sector's rolling statistics to the weekly_rating_df DataFrame
    weekly_rating_df = pd.concat([weekly_rating_df, sector_results])


weekly_rating_df

,sector,end_of_week_date,count,weighted_score_avg
date,,,,
2020-08-28,2025规划,2020-08-28,0.0,0.000000
2020-09-04,2025规划,2020-09-04,0.0,0.000000
2020-09-11,2025规划,2020-09-11,0.0,0.000000
2020-09-18,2025规划,2020-09-18,0.0,0.000000
2020-09-25,2025规划,2020-09-25,0.0,0.000000
...,...,...,...,...
2023-07-28,黑龙江板块,2023-07-28,12.0,5.874757
2023-08-04,黑龙江板块,2023-08-04,13.0,6.656150
2023-08-11,黑龙江板块,2023-08-11,14.0,5.982305


In [ ]:
# groupby([pd.Grouper(key='date', freq='W-FRI'), 'stockCode'])
# sector_data.groupby(['date','stockCode']).size().groupby('date').count().reset_index().groupby([pd.Grouper(key='date', freq='W-FRI')]).sum()
# ss = sector_data.groupby(['date','stockCode','orgCode']).size().groupby('date').count().rolling(window=window_size,min_periods=1).sum()
weekly_rating_df.to_csv('weekly_rating_df.csv',index=False)

### Pricing

In [661]:
loading = False

# Filter based on tag_df stockCode
stock_code_list = tag_df['stockCode'].to_list()

if loading:
    priceDirPath = dirPath + "price/"
    
    # Get the newest date of folders
    for filename in os.listdir(priceDirPath):
        data_date_lst.append(datetime.strptime(filename, '%Y-%m-%d').date())
    date = max(data_date_lst).strftime('%Y-%m-%d')    
    priceDirPath = dirPath + "price/" + date + "/"
    print(priceDirPath)

    # Loading price df
    price_df_list = []
    for stockCode_filename in os.listdir(priceDirPath):
        if stockCode_filename.split('.')[0] in stock_code_list:
            stock_price_df = pd.read_csv(priceDirPath + stockCode_filename, index_col=0)
        price_df_list.append(stock_price_df)
    price_df = pd.concat(price_df_list)

    price_df['日期'] = pd.to_datetime(price_df['日期'])
    # price_df = price_df[price_df['日期']>(price_df['日期'].max()-pd.Timedelta(days=365))]
    price_df["stockCode"] = price_df['stockCode'].astype("str").apply(lambda x: x.zfill(6)).to_list()
    price_df['流通股数'] = price_df['成交量'] * price_df['换手率']
    
    price_df = pd.merge(tag_df, price_df, how='outer',on='stockCode')
else:
    price_df = pd.read_csv
# price_df = price_df[price_df['stockCode'].isin(rating_df['stockCode'])]

data/raw/price/2023-08-26/


In [662]:
price_df

,stockCode,stockName,sector,sectorCount,日期,开盘,收盘,最高,最低,成交量,成交额,振幅,涨跌幅,涨跌额,换手率,流通股数
0,600337,美克家居,2025规划,81,2020-08-26,5.65,5.65,5.74,5.56,274018.0,1.548432e+08,3.19,0.00,0.00,1.59,435688.62
1,600337,美克家居,2025规划,81,2020-08-27,5.62,5.68,5.80,5.52,232458.0,1.315747e+08,4.96,0.53,0.03,1.35,313818.30
2,600337,美克家居,2025规划,81,2020-08-28,5.64,5.62,5.74,5.48,203475.0,1.142536e+08,4.58,-1.06,-0.06,1.18,240100.50
3,600337,美克家居,2025规划,81,2020-08-31,5.63,5.61,5.69,5.57,112466.0,6.334761e+07,2.14,-0.18,-0.01,0.65,73102.90
4,600337,美克家居,2025规划,81,2020-09-01,5.61,5.96,6.02,5.59,326972.0,1.910877e+08,7.66,6.24,0.35,1.89,617977.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39855753,836826,盖世食品,食品饮料,126,2023-08-21,6.03,6.08,6.12,5.97,826.0,4.998031e+05,2.46,-0.33,-0.02,0.16,132.16
39855754,836826,盖世食品,食品饮料,126,2023-08-22,6.06,6.08,6.10,6.00,1516.0,9.162236e+05,1.64,0.00,0.00,0.30,454.80
39855755,836826,盖世食品,食品饮料,126,2023-08-23,6.08,6.06,6.08,6.00,1122.0,6.776579e+05,1.32,-0.33,-0.02,0.22,246.84
39855756,836826,盖世食品,食品饮料,126,2023-08-24,6.06,6.13,6.14,5.95,2279.0,1.382498e+06,3.14,1.16,0.07,0.45,1025.55


In [663]:
price_df = price_df.rename(columns={"开盘":'open','收盘':'close','成交量':'volume','成交额':'turnover',"最高":'high',"最低":'low','流通股数':'outstanding'})
price_df = price_df.groupby(['日期','sector']).agg({
        'close': lambda x: x.mean(),  
        'open': lambda x: x.mean(),
        'high': lambda x: x.mean(),  
        'low': lambda x: x.mean(),
        'volume': lambda x: sum(x),
        'turnover': lambda x: sum(x),
        'outstanding': lambda x: sum(x)
})

In [671]:
price_df.reset_index(inplace=True)
price_df = price_df.rename(columns={'日期':'date'})

### Merged

In [673]:
df = price_df.reset_index()
df = df.sort_values(by=['sector', 'date'])
df['log_return'] = df.groupby('sector')['close'].apply(lambda x: x / x.shift(1)).apply(lambda x: pd.np.log(x))

df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
# weekly_data = df.groupby('sector').resample('W').agg({
#     '开盘': 'mean', '收盘': 'mean',
#     '成交量': 'sum', '成交额': 'sum', '流通股数': 'mean',
#     '最高': 'max', '最低': 'min'})
# weekly_data = weekly_data.rename(columns={"开盘":'open','收盘':'close','成交量':'volume','成交额':'turnover',"最高":'high',"最低":'low','流通股数':'outstanding'})

weekly_data = df.groupby('sector').resample('W-FRI').agg({
    'open': 'mean', 'close': 'mean',
    'volume': 'sum', 'turnover': 'sum', 'outstanding': 'mean',
    'high': 'max', 'low': 'min'})

weekly_data['log_return'] = weekly_data.groupby('sector')['close'].apply(lambda x: x / x.shift(1)).apply(lambda x: pd.np.log(x))
weekly_data['log_return'] = weekly_data.groupby('sector')['log_return'].shift(-1)

data = weekly_data
data.reset_index(inplace=True)

In [674]:
from pysrc import utils

In [676]:
df.drop(columns=['level_0','index'],inplace=True)

In [678]:
weekly_rating_df

""


In [677]:
df = pd.merge(weekly_rating_df.reset_index(),weekly_data,on=['date','sector'],how='outer')
df = df.rename(columns={"weighted_score_avg":"emRatingValue"}).drop(columns=["end_of_week_date"])
df = df[df['log_return'].notna()]
df = pd.merge(df,rating_df.groupby('sector')['stockCode'].apply(lambda x:len(set(x))).reset_index().rename(columns={'stockCode':'stockCount'}),on='sector',how='left')
df['count'] = df['count']/df['stockCount']
df['emRatingValue'] = df['emRatingValue']/df['stockCount']
df.drop(columns=['stockCount'])
data = df

cols = data.columns.to_list()
y_cols = ['log_return']
x_cols = [col for col in cols if col not in ['log_return','date','sector']]
x_cols + y_cols

KeyError: 'date'

In [646]:
diff_periods = [1, 3]  # Number of lag periods
for diff in diff_periods:
    for col in x_cols:
        data[f'{col}_diff_{diff}'] = data.groupby('sector')[col].diff(diff)


In [647]:
lag_periods = [1, 3]  # Number of lag periods
for lag in lag_periods:
    for col in x_cols:
        data[f'{col}_lag_{lag}'] = data.groupby('sector')[col].shift(lag)


In [648]:
moving_averages = [1, 3]  # Moving average periods
for ma in moving_averages:
    for col in x_cols:
        data[f'{col}_ma_{ma}'] = data.groupby('sector')[col].rolling(window=ma).mean().reset_index(level='sector',drop=True)


In [649]:
data.dropna(inplace=True)
# data = data[data['log_return'].notna()]
data.reset_index(inplace=True)
data.drop(columns='index',inplace=True)
data.to_csv("featured_price.csv",index=False)


In [651]:
summary(data[['date','log_return']])

data shape: (45444, 2)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
date,datetime64[ns],0,0.0,92,NaN,NaN,2021-09-10 00:00:00,2021-09-17 00:00:00,2021-09-24 00:00:00
log_return,float64,0,0.0,45441,-0.308809,0.428567,-0.018027,0.006946,-0.020991


1

### Archive

In [ ]:
df.sort_values()

In [ ]:
summary(df)

In [ ]:
df = df.sort_values(by=['sector', '日期'])
df['log_return'] = df.groupby('sector')['close'].apply(lambda x: x / x.shift(1)).apply(lambda x: pd.np.log(x))

In [ ]:
df

In [ ]:
df['date'] = pd.to_datetime(df['日期'])
df.set_index('date', inplace=True)
# weekly_data = df.groupby('sector').resample('W').agg({
#     '开盘': 'mean', '收盘': 'mean',
#     '成交量': 'sum', '成交额': 'sum', '流通股数': 'mean',
#     '最高': 'max', '最低': 'min'})
# weekly_data = weekly_data.rename(columns={"开盘":'open','收盘':'close','成交量':'volume','成交额':'turnover',"最高":'high',"最低":'low','流通股数':'outstanding'})

weekly_data = df.groupby('sector').resample('W-FRI').agg({
    'open': 'mean', 'close': 'mean',
    'volume': 'sum', 'turnover': 'sum', 'outstanding': 'mean',
    'high': 'max', 'low': 'min'})

weekly_data['log_return'] = weekly_data.groupby('sector')['close'].apply(lambda x: x / x.shift(1)).apply(lambda x: pd.np.log(x))
weekly_data['log_return'] = weekly_data.groupby('sector')['log_return'].shift(-1)

data = weekly_data
data.reset_index(inplace=True)

In [ ]:
data[data['sector']=='2025规划'][-20:]

In [ ]:
weekly_rating_df[weekly_rating_df['sector']=='2025规划'].sort_values('date')[-20:]

In [ ]:
# Create lagged features
lag_periods = [1, 5, 10]  # Number of lag periods
for lag in lag_periods:
    for col in ['open', 'close', 'volume', 'turnover', 'outstanding', 'high', 'low']:
        data[f'{col}_lag_{lag}'] = data.groupby('sector')[col].shift(lag)


In [ ]:
# Reset the index to separate 'sector' from the index
data.reset_index(level='sector', inplace=True)

# Create moving averages
moving_averages = [5, 10, 20]  # Moving average periods
for ma in moving_averages:
    for col in ['open', 'close', 'volume', 'turnover', 'outstanding', 'high', 'low']:
        data[f'{col}_ma_{ma}'] = data.groupby('sector')[col].rolling(window=ma).mean().reset_index(level='sector', drop=True)


In [ ]:

# Calculate price change features
data['price_change'] = data.groupby('sector')['close'].diff()

# Calculate log return features
data['log_return_lag_1'] = data.groupby('sector')['log_return'].shift(1)

# Calculate relative indicators
data['return_vs_lag'] = data['log_return'] - data['log_return_lag_1']
data['return_vs_ma_5'] = data['log_return'] - data['close_ma_5']
data['return_vs_ma_10'] = data['log_return'] - data['close_ma_10']

# Drop rows with NaN due to feature engineering
data.dropna(inplace=True)

In [ ]:
data

In [ ]:
data = data.reset_index()
data.to_csv("featured_price.csv",index=False)